## result collect resize
- cycleGAN을 학습 시킨 뒤 scene별 이미지들을 생성하기 위한 코드입니다.
- 생성된 이미지들은 다시 yolov5모델에 학습하기 위해 원본 크기(1242, 375)로 resize과정을 거칩니다.

In [9]:
import os
import shutil
from PIL import Image
from IPython.display import Image as Img
from IPython.display import display
import cv2

generated_result_path = r'C:\Users\junho\Desktop\cycleGAN\pytorch-CycleGAN-and-pix2pix\results\overcast_cyclegan\test_latest\images'
saving_path = r'C:\Users\junho\Desktop\cycleGAN\dataset4yolov5'


In [15]:
shutil.rmtree(generated_result_path)

### test 명령어
- 아래의 명령어를 통해 학습된 cyclegan 모델의 출력결과를 확인할 수 있습니다.
    ```
    python test.py --dataroot C:\Users\junho\Desktop\cycleGAN\vkitti_2.0.3_rgb\Scene18\clone\frames\rgb --name overcast_cyclegan --model cycle_gan --preprocess scale_width --load_size 1242 --num_test 1000
    ```

In [16]:
# 필요한 testB 이미지만 saving_path로 가져오기
# 가져온 이미지 vkitti 원본크기(1242, 375)로 resize

idx = '200700' # scene01 02 06 18 20
img_list = os.listdir(generated_result_path)
i=0
for file in img_list:
    if file.endswith('fake_B.png'):
        shutil.copy(generated_result_path+'/'+file, 
                    saving_path+'/'+idx+'{}'.format(i).zfill(6)+'.png')
        image = Image.open(saving_path+'/'+idx+'{}'.format(i).zfill(6)+'.png')
        resize_image = image.resize((1242, 375))
        resize_image.save(saving_path+'/'+idx+'{}'.format(i).zfill(6)+'.png')
        i+=1

### 결과 시각화

- cycleGAN의 결과를 gif, mp4의 방식으로 시각화해볼 수 있습니다.

In [17]:
# result gif 생성
def generate_gif(path,name):
    img_list = os.listdir(path)
    img_list = [path + '/' + x for x in img_list]
    images = [Image.open(x) for x in img_list]

    im = images[0]
    im.save(name, save_all=True, append_images=images[1:],loop=0, duration=1)
    return Img(url=name)

gif = generate_gif(saving_path,'out.gif')
display(gif)


In [13]:
# result.mp4
def img2mp4(paths, pathOut, fps=10):
    frame_array = []
    for idx, path in enumerate(paths):
        img = cv2.imread(path)
        height, width, layers = img.shape
        size = (width, height)
        frame_array.append(img)
    out = cv2.VideoWriter(pathOut, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

paths = sorted(os.listdir(saving_path))
paths = [os.path.join(saving_path, path) for path in paths]
img2mp4(paths , r'C:\Users\junho\Desktop\cycleGAN\out.mp4', fps=24) 


In [15]:
# raw vkitti.mp4 (scene01)
vkitti_path = r'C:\Users\junho\Desktop\cycleGAN\vkitti_2.0.3_rgb\Scene01\clone\frames\rgb\testA'

paths = sorted(os.listdir(vkitti_path))
paths = [os.path.join(vkitti_path, path) for path in paths]
img2mp4(paths, r'C:\Users\junho\Desktop\cycleGAN\raw_vkitti.mp4', fps=24)


In [18]:
# raw vkitti gif (scene01)
gif = generate_gif(vkitti_path,'default_vkitti.gif')
display(gif)
